# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""82409298610024…",1.2927e19,"""15921522927488…","""../../../../da…","""12193965033264…","""15389257823873…",false
"""27698138475280…",1.4856e19,"""16914983411326…","""../../../../da…","""12193965033264…","""17996305109511…",true
"""68461758763726…",5.6458e17,"""33519528195602…","""../../../../da…","""63570195832470…","""15389257823873…",false
"""15867902113822…",4.0723e18,"""16914983411326…","""../../../../da…","""14626966501206…","""15389257823873…",false
"""11968569675991…",1.4677e19,"""15921522927488…","""../../../../da…","""14626966501206…","""14246675187620…",true
"""96493690655816…",1.6330e19,"""33519528195602…","""../../../../da…","""15420909341890…",null,false
"""57822697402185…",8.7675e18,"""33519528195602…","""../../../../da…","""14626966501206…","""14246675187620…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""12441179652233…",1.2927e19,"""63794180391804…","""../../../../da…","""FirstMortgage3…","""12607999027858…","""Rejected"""
"""19606881662817…",1.4856e19,"""20494658842556…","""../../../../da…","""FirstMortgage3…","""64213579172956…","""Accepted"""
"""51345578829651…",5.6458e17,"""14808159721085…","""../../../../da…","""MoneyMarketSav…","""12607999027858…","""Rejected"""
"""84260137127847…",4.0723e18,"""20494658842556…","""../../../../da…","""BasicChecking""","""12607999027858…","""Rejected"""
"""14435410012424…",1.4677e19,"""63794180391804…","""../../../../da…","""BasicChecking""","""17859243384144…","""Accepted"""
"""61644790185119…",1.6330e19,"""14808159721085…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""50192412674063…",8.7675e18,"""14808159721085…","""../../../../da…","""BasicChecking""","""17859243384144…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""57168702719767…",1.2927e19,"""15648645790562…","""../../../../da…","""FirstMortgage3…","""17585784817063…","""Rejected"""
"""11149485785138…",1.4856e19,"""10798002436636…","""../../../../da…","""FirstMortgage3…","""15423264842149…","""Accepted"""
"""14994428720821…",5.6458e17,"""89951493829927…","""../../../../da…","""MoneyMarketSav…","""17585784817063…","""Rejected"""
"""15251952415935…",4.0723e18,"""10798002436636…","""../../../../da…","""BasicChecking""","""17585784817063…","""Rejected"""
"""33012906122521…",1.4677e19,"""15648645790562…","""../../../../da…","""BasicChecking""","""12712649283070…","""Accepted"""
"""78478600415907…",1.6330e19,"""89951493829927…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""77555486655392…",8.7675e18,"""89951493829927…","""../../../../da…","""BasicChecking""","""12712649283070…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""40202338202199…",1.2927e19,"""17242762668245…","""FirstMortgage3…","""15747165217601…","""Rejected"""
"""17225155306564…",1.4856e19,"""12278239919267…","""FirstMortgage3…","""49892653629190…","""Accepted"""
"""11727372901717…",5.6458e17,"""80427167448238…","""MoneyMarketSav…","""15747165217601…","""Rejected"""
"""14441511634725…",4.0723e18,"""12278239919267…","""BasicChecking""","""15747165217601…","""Rejected"""
"""99178725444057…",1.4677e19,"""17242762668245…","""BasicChecking""","""28619093686412…","""Accepted"""
"""12853391097906…",1.6330e19,"""80427167448238…","""UPlusFinPerson…",null,"""Rejected"""
"""49628486778868…",8.7675e18,"""80427167448238…","""BasicChecking""","""28619093686412…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'